<a href="https://colab.research.google.com/github/seiya-iiyama/tutorial/blob/main/%E3%82%B9%E3%82%BF%E3%83%BC%E3%83%80%E3%82%B9%E3%83%88_%E3%82%B3%E3%83%B3%E3%83%86%E3%83%B3%E3%83%84%E8%A6%96%E8%81%B4%E3%83%AC%E3%83%9D%E3%83%BC%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SET UP

In [ ]:
import pandas as pd
import pandas_gbq

from google.colab import files
from google.colab.data_table import DataTable
DataTable.max_columns = 100

from io import StringIO

In [ ]:
# 設定一覧
conf = {
'file_name': '20250115',    #必ず半角英数で指定、作業日に変更
'previous_month_start_date': '2024-12-01', # 前月の1日に変更
'previous_month_end_date': '2024-12-31', # 前月の末日に変更
'current_month_start_date': '2025-01-01', # 今月の1日に変更
'age_base_date': '2024-03-01',  # 年度が変わったら更新する
}
conf['previous_month_start_date_nodash'] = conf['previous_month_start_date'].replace("-", "")
conf['previous_month_end_date_nodash'] = conf['previous_month_end_date'].replace("-", "")
conf['current_month_start_date_nodash'] = conf['current_month_start_date'].replace("-", "")

## 実行クエリ_超特急

In [ ]:
conf['group_name'] = 'bullettrain'

In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

fc_mart AS (
SELECT
  system_id,
  sex,
  DATE(join_date) AS join_date,
  DATE_TRUNC(DATE(join_date), YEAR) AS join_year,
  birthday,
  CASE --年代別
    WHEN sex = '99.その他' THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) < 0 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) > 100 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 11 THEN 'a.~11'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 14 THEN 'b.12~14'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 17 THEN 'c.15~17'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 21 THEN 'd.18~21'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 29 THEN 'e.22~29'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 39 THEN 'f.30~39'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 49 THEN 'g.40~49'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) >= 50 THEN 'h.50~'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) IS NULL THEN 'z.不明'
    ELSE 'zz.Other'
  END AS age_kubun,
FROM
  `gree-dionysus-stardust.production_sdr.process_fc_{group_name}_{current_month_start_date_nodash}`
WHERE
  DATE(join_date) <= '{previous_month_end_date}'
  AND DATE(membership_expiration_date) >= '{current_month_start_date}'
)

SELECT
  sex,
  age_kubun,
--  join_year,
  COUNT(DISTINCT(fc_mart.system_id)) AS fc_uu,
  COUNT(DISTINCT(CASE WHEN DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}' THEN content_dl.user_id ELSE NULL END)) AS content_uu,
FROM
  fc_mart
FULL OUTER JOIN
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl ON fc_mart.system_id = content_dl.user_id
JOIN
  content_master USING(file_id)
--WHERE
--  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  1,2

""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_zokusei = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_zokusei.to_csv('df_zokusei_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_zokusei

<ipython-input-15-8d804f06b04e>:74: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_bq_taizai = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,sex,age_kubun,fc_uu,content_uu
0,None,None,0,15
1,1.女性,a.~11,53,10
2,1.女性,b.12~14,94,31
3,1.女性,c.15~17,580,234
4,1.女性,d.18~21,2311,785
5,1.女性,e.22~29,4827,1177
6,1.女性,f.30~39,1760,589
7,1.女性,g.40~49,2139,1014
8,1.女性,h.50~,2257,1018
9,2.男性,a.~11,23,2


In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

fc_mart AS (
SELECT
  system_id,
  sex,
  DATE(join_date) AS join_date,
  DATE_TRUNC(DATE(join_date), YEAR) AS join_year,
  birthday,
  CASE --年代別
    WHEN sex = '99.その他' THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) < 0 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) > 100 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 11 THEN 'a.~11'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 14 THEN 'b.12~14'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 17 THEN 'c.15~17'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 21 THEN 'd.18~21'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 29 THEN 'e.22~29'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 39 THEN 'f.30~39'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 49 THEN 'g.40~49'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) >= 50 THEN 'h.50~'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) IS NULL THEN 'z.不明'
    ELSE 'zz.Other'
  END AS age_kubun,
FROM
  `gree-dionysus-stardust.production_sdr.process_fc_{group_name}_{current_month_start_date_nodash}`
WHERE
  DATE(join_date) <= '{previous_month_end_date}'
  AND DATE(membership_expiration_date) >= '{previous_month_start_date}'
)

SELECT
  -- sex,
  -- age_kubun,
  join_year,
  COUNT(DISTINCT(fc_mart.system_id)) AS fc_uu,
  COUNT(DISTINCT(CASE WHEN DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}' THEN content_dl.user_id ELSE NULL END)) AS content_uu,
FROM
  fc_mart
FULL OUTER JOIN
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl ON fc_mart.system_id = content_dl.user_id
JOIN
  content_master USING(file_id)
--WHERE
--  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  1

""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_join_year = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_join_year.to_csv('df_join_year_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_join_year

<ipython-input-16-c421e090bf5c>:74: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_join_year = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,join_year,fc_uu,content_uu
0,NaT,0,0
1,2014-01-01,383,70
2,2015-01-01,165,30
3,2016-01-01,130,20
4,2017-01-01,270,58
5,2018-01-01,736,169
6,2019-01-01,859,219
7,2020-01-01,1574,539
8,2021-01-01,3165,1109
9,2022-01-01,2105,725


In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

content_month_master AS (
SELECT
  *,
  ROW_NUMBER() OVER(ORDER BY cnt_dl DESC) AS dl_rank_in_month,
FROM (
  SELECT
    file_id,
  COUNT(DISTINCT(id)) AS cnt_dl,
  FROM
    `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
  WHERE
    DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
  GROUP BY
    ALL
  )
)

SELECT
--  DATE(downloaded_date) AS dl_date,
/*
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN dl_rank_in_month ELSE 99 END AS dl_rank_in_month,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN file_id ELSE 'other' END AS file_id,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_type ELSE 'other' END AS content_type,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_name ELSE 'other' END AS content_name,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.min_dl_date ELSE NULL END AS min_dl_date,
*/
  COUNT(DISTINCT(user_id)) AS cnt_user,
  COUNT(DISTINCT(id)) AS cnt_dl,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
JOIN
  content_master USING(file_id)
JOIN
  content_month_master USING(file_id)
WHERE
  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  1

""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_total = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_total.to_csv('df_total_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_total

<ipython-input-17-eda9e0f3f462>:67: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_total = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,cnt_user,cnt_dl
0,5071,32465


In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

content_month_master AS (
SELECT
  *,
  ROW_NUMBER() OVER(ORDER BY cnt_dl DESC) AS dl_rank_in_month,
FROM (
  SELECT
    file_id,
  COUNT(DISTINCT(id)) AS cnt_dl,
  FROM
    `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
  WHERE
    DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
  GROUP BY
    ALL
  )
)

SELECT
 DATE(downloaded_date) AS dl_date,
/*
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN dl_rank_in_month ELSE 99 END AS dl_rank_in_month,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN file_id ELSE 'other' END AS file_id,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_type ELSE 'other' END AS content_type,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_name ELSE 'other' END AS content_name,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.min_dl_date ELSE NULL END AS min_dl_date,
*/
  COUNT(DISTINCT(user_id)) AS cnt_user,
  COUNT(DISTINCT(id)) AS cnt_dl,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
JOIN
  content_master USING(file_id)
JOIN
  content_month_master USING(file_id)
WHERE
  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  1

""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_dl_date = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_dl_date.to_csv('df_dl_date_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_dl_date

<ipython-input-18-a568901f8d88>:67: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_dl_date = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,dl_date,cnt_user,cnt_dl
0,2024-11-01,724,2058
1,2024-11-02,1085,2177
2,2024-11-03,1021,2941
3,2024-11-04,541,1314
4,2024-11-05,329,996
5,2024-11-06,356,926
6,2024-11-07,217,715
7,2024-11-08,170,622
8,2024-11-09,159,648
9,2024-11-10,189,744


In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

content_month_master AS (
SELECT
  *,
  ROW_NUMBER() OVER(ORDER BY cnt_dl DESC) AS dl_rank_in_month,
FROM (
  SELECT
    file_id,
  COUNT(DISTINCT(id)) AS cnt_dl,
  FROM
    `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
  WHERE
    DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
  GROUP BY
    ALL
  )
)

SELECT
 DATE(downloaded_date) AS dl_date,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN dl_rank_in_month ELSE 99 END AS dl_rank_in_month,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_name ELSE 'other' END AS content_name,
/*
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_type ELSE 'other' END AS content_type,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN file_id ELSE 'other' END AS file_id,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.min_dl_date ELSE NULL END AS min_dl_date,
  COUNT(DISTINCT(user_id)) AS cnt_user,
*/
  COUNT(DISTINCT(id)) AS cnt_dl,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
JOIN
  content_master USING(file_id)
JOIN
  content_month_master USING(file_id)
WHERE
  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  1,2

""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_dl_rank = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_dl_rank.to_csv('df_dl_rank_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_dl_rank

<ipython-input-19-b29128674315>:67: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_dl_rank = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,dl_date,dl_rank_in_month,content_name,cnt_dl
0,2024-11-01,5,橘柊生バースデー動画(？)『DISH//'s BBQ』,75
1,2024-11-01,6,2024/10/27(日)北村匠海 監督映画「世界征服やめた」ワールドプレミア上映イベント,113
2,2024-11-01,7,2024.November,539
3,2024-11-01,8,2024.November,319
4,2024-11-01,9,2024/10/26(土)DJ To-i「SPOOKY PUMPKIN 2024 ～PURO...,58
...,...,...,...,...
258,2024-11-30,7,2024.November,16
259,2024-11-30,8,2024.November,6
260,2024-11-30,9,2024/10/26(土)DJ To-i「SPOOKY PUMPKIN 2024 ～PURO...,6
261,2024-11-30,10,【後編】泉大智バースデー動画『DISH//'sキッチン』,20


In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

fc_mart AS (
SELECT
  system_id,
  sex,
  DATE(join_date) AS join_date,
  DATE_TRUNC(DATE(join_date), YEAR) AS join_year,
  birthday,
  CASE --年代別
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) < 0 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) > 100 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 11 THEN 'a.~11'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 14 THEN 'b.12~14'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 17 THEN 'c.15~17'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 21 THEN 'd.18~21'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 29 THEN 'e.22~29'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 39 THEN 'f.30~39'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 49 THEN 'g.40~49'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) >= 50 THEN 'h.50~'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) IS NULL THEN 'z.不明'
    ELSE 'zz.Other'
  END AS age_kubun,
FROM
  `gree-dionysus-stardust.production_sdr.process_fc_{group_name}_{current_month_start_date_nodash}`
WHERE
  DATE(join_date) <= '{previous_month_end_date}'
  AND DATE(membership_expiration_date) >= '{previous_month_start_date}'
)

SELECT
  file_id,
  content_master.content_type,
  content_master.content_name,
  content_master.min_dl_date,
  COUNT(DISTINCT(user_id)) AS cnt_user,
  COUNT(DISTINCT(id)) AS cnt_dl,
--性年代別
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'a.~11' THEN user_id ELSE NULL END)) AS uu_women_11,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'b.12~14' THEN user_id ELSE NULL END)) AS uu_women_12_14,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'c.15~17' THEN user_id ELSE NULL END)) AS uu_women_15_17,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'd.18~21' THEN user_id ELSE NULL END)) AS uu_women_18_21,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'e.22~29' THEN user_id ELSE NULL END)) AS uu_women_22_29,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'f.30~39' THEN user_id ELSE NULL END)) AS uu_women_30_39,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'g.40~49' THEN user_id ELSE NULL END)) AS uu_women_40_49,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'h.50~' THEN user_id ELSE NULL END)) AS uu_women_50_,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'a.~11' THEN user_id ELSE NULL END)) AS uu_women_11,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'b.12~14' THEN user_id ELSE NULL END)) AS uu_men_12_14,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'c.15~17' THEN user_id ELSE NULL END)) AS uu_men_15_17,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'd.18~21' THEN user_id ELSE NULL END)) AS uu_men_18_21,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'e.22~29' THEN user_id ELSE NULL END)) AS uu_men_22_29,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'f.30~39' THEN user_id ELSE NULL END)) AS uu_men_30_39,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'g.40~49' THEN user_id ELSE NULL END)) AS uu_men_40_49,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'h.50~' THEN user_id ELSE NULL END)) AS uu_men_50_,
  COUNT(DISTINCT(CASE WHEN sex NOT IN ('1.女性' ,'2.男性') OR age_kubun NOT LIKE '%.%' THEN user_id ELSE NULL END)) AS uu_otherage,
  COUNT(DISTINCT(CASE WHEN sex IS NULL OR age_kubun IS NULL THEN user_id ELSE NULL END)) AS uu_notFC,
  --FC入会年
  COUNT(DISTINCT(CASE WHEN join_year = '2014-01-01' THEN user_id ELSE NULL END)) AS uu_2014join,
  COUNT(DISTINCT(CASE WHEN join_year = '2015-01-01' THEN user_id ELSE NULL END)) AS uu_2015join,
  COUNT(DISTINCT(CASE WHEN join_year = '2016-01-01' THEN user_id ELSE NULL END)) AS uu_2016join,
  COUNT(DISTINCT(CASE WHEN join_year = '2017-01-01' THEN user_id ELSE NULL END)) AS uu_2017join,
  COUNT(DISTINCT(CASE WHEN join_year = '2018-01-01' THEN user_id ELSE NULL END)) AS uu_2018join,
  COUNT(DISTINCT(CASE WHEN join_year = '2019-01-01' THEN user_id ELSE NULL END)) AS uu_2019join,
  COUNT(DISTINCT(CASE WHEN join_year = '2020-01-01' THEN user_id ELSE NULL END)) AS uu_2020join,
  COUNT(DISTINCT(CASE WHEN join_year = '2021-01-01' THEN user_id ELSE NULL END)) AS uu_2021join,
  COUNT(DISTINCT(CASE WHEN join_year = '2022-01-01' THEN user_id ELSE NULL END)) AS uu_2022join,
  COUNT(DISTINCT(CASE WHEN join_year = '2023-01-01' THEN user_id ELSE NULL END)) AS uu_2023join,
  COUNT(DISTINCT(CASE WHEN join_year = '2024-01-01' THEN user_id ELSE NULL END)) AS uu_2024join,
  COUNT(DISTINCT(CASE WHEN join_year = '2025-01-01' THEN user_id ELSE NULL END)) AS uu_2025join,
  COUNT(DISTINCT(CASE WHEN NOT join_year BETWEEN '2014-01-01' AND '2025-01-01'THEN user_id ELSE NULL END)) AS uu_otherjoin,
  COUNT(DISTINCT(CASE WHEN join_year IS NULL THEN user_id ELSE NULL END)) AS uu_notFC,
FROM
  fc_mart
FULL OUTER JOIN
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl ON fc_mart.system_id = content_dl.user_id
JOIN
  content_master USING(file_id)
WHERE
  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  cnt_dl DESC
LIMIT
  20


""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_detail = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_detail.to_csv('df_detail_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_detail

<ipython-input-20-02c317ea8809>:110: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,file_id,content_type,content_name,min_dl_date,cnt_user,cnt_dl,uu_women_11,uu_women_12_14,uu_women_15_17,uu_women_18_21,...,uu_2017join,uu_2018join,uu_2019join,uu_2020join,uu_2021join,uu_2022join,uu_2023join,uu_2024join,uu_otherjoin,uu_notFC_1
0,c77b2732a7d3eec3d5e4b47b009335d40dc430e6e6bdf0...,新ムービー,2024/11/2(土)日本テレビ「with MUSIC」,2024-11-02,2663,4369,3,13,95,285,...,17,64,92,274,646,364,815,354,0,0
1,A1008533100000002642,フォト,DISH// ARENA LIVE 2023「HAPPY？」DAY2,2024-11-12,2145,3876,2,18,116,293,...,7,46,73,196,479,332,724,261,0,0
2,e8613e0c54c440578102994086b733bab01f90444e1a88...,新ムービー,2024/11/30(土)DISH// ARENA TOUR 2024-2025「群青飛行」...,2024-11-30,2228,2852,1,10,97,323,...,37,77,100,246,479,324,677,229,0,0
3,A1008533100000002643,フォト,DISH// ARENA LIVE 2023「HAPPY？」DAY1,2024-11-13,1335,2029,0,9,70,167,...,8,28,41,117,265,201,471,193,0,0
4,16460a2a1802504811d3a3030fbf7995f2694ec0ad031f...,新ムービー,橘柊生バースデー動画(？)『DISH//'s BBQ』,2024-10-15,1094,1668,2,5,45,179,...,10,40,47,131,261,129,307,149,0,0
5,A1008533100000002636,フォト,2024/10/27(日)北村匠海 監督映画「世界征服やめた」ワールドプレミア上映イベント,2024-10-28,1032,1309,4,7,58,112,...,3,9,13,71,210,149,387,186,0,0
6,A1008530100000002638,PC壁紙,2024.November,2024-11-01,676,978,2,10,21,63,...,0,14,18,63,160,99,224,90,0,0
7,A1008530100000002639,PC壁紙,2024.November,2024-11-01,454,564,1,5,15,63,...,1,12,14,41,96,73,149,58,0,0
8,A1008533100000002637,フォト,2024/10/26(土)DJ To-i「SPOOKY PUMPKIN 2024 ～PURO...,2024-10-28,479,539,2,5,32,67,...,1,4,10,32,87,72,189,83,0,0
9,transfar3118,新ムービー,【後編】泉大智バースデー動画『DISH//'sキッチン』,2024-07-24,219,384,0,1,11,34,...,4,9,13,23,52,27,49,35,0,0


## 実行クエリ_DISH//

In [ ]:
conf['group_name'] = 'dish'

In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

fc_mart AS (
SELECT
  system_id,
  sex,
  DATE(join_date) AS join_date,
  DATE_TRUNC(DATE(join_date), YEAR) AS join_year,
  birthday,
  CASE --年代別
    WHEN sex = '99.その他' THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) < 0 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) > 100 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 11 THEN 'a.~11'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 14 THEN 'b.12~14'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 17 THEN 'c.15~17'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 21 THEN 'd.18~21'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 29 THEN 'e.22~29'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 39 THEN 'f.30~39'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 49 THEN 'g.40~49'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) >= 50 THEN 'h.50~'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) IS NULL THEN 'z.不明'
    ELSE 'zz.Other'
  END AS age_kubun,
FROM
  `gree-dionysus-stardust.production_sdr.process_fc_{group_name}_{current_month_start_date_nodash}`
WHERE
  DATE(join_date) <= '{previous_month_end_date}'
  AND DATE(membership_expiration_date) >= '{current_month_start_date}'
)

SELECT
  sex,
  age_kubun,
--  join_year,
  COUNT(DISTINCT(fc_mart.system_id)) AS fc_uu,
  COUNT(DISTINCT(CASE WHEN DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}' THEN content_dl.user_id ELSE NULL END)) AS content_uu,
FROM
  fc_mart
FULL OUTER JOIN
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl ON fc_mart.system_id = content_dl.user_id
JOIN
  content_master USING(file_id)
--WHERE
--  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  1,2

""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_zokusei = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_zokusei.to_csv('df_zokusei_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_zokusei

<ipython-input-15-8d804f06b04e>:74: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_bq_taizai = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,sex,age_kubun,fc_uu,content_uu
0,None,None,0,15
1,1.女性,a.~11,53,10
2,1.女性,b.12~14,94,31
3,1.女性,c.15~17,580,234
4,1.女性,d.18~21,2311,785
5,1.女性,e.22~29,4827,1177
6,1.女性,f.30~39,1760,589
7,1.女性,g.40~49,2139,1014
8,1.女性,h.50~,2257,1018
9,2.男性,a.~11,23,2


In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

fc_mart AS (
SELECT
  system_id,
  sex,
  DATE(join_date) AS join_date,
  DATE_TRUNC(DATE(join_date), YEAR) AS join_year,
  birthday,
  CASE --年代別
    WHEN sex = '99.その他' THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) < 0 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) > 100 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 11 THEN 'a.~11'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 14 THEN 'b.12~14'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 17 THEN 'c.15~17'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 21 THEN 'd.18~21'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 29 THEN 'e.22~29'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 39 THEN 'f.30~39'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 49 THEN 'g.40~49'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) >= 50 THEN 'h.50~'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) IS NULL THEN 'z.不明'
    ELSE 'zz.Other'
  END AS age_kubun,
FROM
  `gree-dionysus-stardust.production_sdr.process_fc_{group_name}_{current_month_start_date_nodash}`
WHERE
  DATE(join_date) <= '{previous_month_end_date}'
  AND DATE(membership_expiration_date) >= '{previous_month_start_date}'
)

SELECT
  -- sex,
  -- age_kubun,
  join_year,
  COUNT(DISTINCT(fc_mart.system_id)) AS fc_uu,
  COUNT(DISTINCT(CASE WHEN DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}' THEN content_dl.user_id ELSE NULL END)) AS content_uu,
FROM
  fc_mart
FULL OUTER JOIN
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl ON fc_mart.system_id = content_dl.user_id
JOIN
  content_master USING(file_id)
--WHERE
--  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  1

""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_join_year = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_join_year.to_csv('df_join_year_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_join_year

<ipython-input-16-c421e090bf5c>:74: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_join_year = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,join_year,fc_uu,content_uu
0,NaT,0,0
1,2014-01-01,383,70
2,2015-01-01,165,30
3,2016-01-01,130,20
4,2017-01-01,270,58
5,2018-01-01,736,169
6,2019-01-01,859,219
7,2020-01-01,1574,539
8,2021-01-01,3165,1109
9,2022-01-01,2105,725


In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

content_month_master AS (
SELECT
  *,
  ROW_NUMBER() OVER(ORDER BY cnt_dl DESC) AS dl_rank_in_month,
FROM (
  SELECT
    file_id,
  COUNT(DISTINCT(id)) AS cnt_dl,
  FROM
    `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
  WHERE
    DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
  GROUP BY
    ALL
  )
)

SELECT
--  DATE(downloaded_date) AS dl_date,
/*
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN dl_rank_in_month ELSE 99 END AS dl_rank_in_month,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN file_id ELSE 'other' END AS file_id,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_type ELSE 'other' END AS content_type,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_name ELSE 'other' END AS content_name,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.min_dl_date ELSE NULL END AS min_dl_date,
*/
  COUNT(DISTINCT(user_id)) AS cnt_user,
  COUNT(DISTINCT(id)) AS cnt_dl,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
JOIN
  content_master USING(file_id)
JOIN
  content_month_master USING(file_id)
WHERE
  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  1

""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_total = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_total.to_csv('df_total_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_total

<ipython-input-17-eda9e0f3f462>:67: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_total = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,cnt_user,cnt_dl
0,5071,32465


In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

content_month_master AS (
SELECT
  *,
  ROW_NUMBER() OVER(ORDER BY cnt_dl DESC) AS dl_rank_in_month,
FROM (
  SELECT
    file_id,
  COUNT(DISTINCT(id)) AS cnt_dl,
  FROM
    `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
  WHERE
    DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
  GROUP BY
    ALL
  )
)

SELECT
 DATE(downloaded_date) AS dl_date,
/*
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN dl_rank_in_month ELSE 99 END AS dl_rank_in_month,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN file_id ELSE 'other' END AS file_id,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_type ELSE 'other' END AS content_type,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_name ELSE 'other' END AS content_name,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.min_dl_date ELSE NULL END AS min_dl_date,
*/
  COUNT(DISTINCT(user_id)) AS cnt_user,
  COUNT(DISTINCT(id)) AS cnt_dl,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
JOIN
  content_master USING(file_id)
JOIN
  content_month_master USING(file_id)
WHERE
  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  1

""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_dl_date = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_dl_date.to_csv('df_dl_date_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_dl_date

<ipython-input-18-a568901f8d88>:67: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_dl_date = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,dl_date,cnt_user,cnt_dl
0,2024-11-01,724,2058
1,2024-11-02,1085,2177
2,2024-11-03,1021,2941
3,2024-11-04,541,1314
4,2024-11-05,329,996
5,2024-11-06,356,926
6,2024-11-07,217,715
7,2024-11-08,170,622
8,2024-11-09,159,648
9,2024-11-10,189,744


In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

content_month_master AS (
SELECT
  *,
  ROW_NUMBER() OVER(ORDER BY cnt_dl DESC) AS dl_rank_in_month,
FROM (
  SELECT
    file_id,
  COUNT(DISTINCT(id)) AS cnt_dl,
  FROM
    `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
  WHERE
    DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
  GROUP BY
    ALL
  )
)

SELECT
 DATE(downloaded_date) AS dl_date,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN dl_rank_in_month ELSE 99 END AS dl_rank_in_month,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_name ELSE 'other' END AS content_name,
/*
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.content_type ELSE 'other' END AS content_type,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN file_id ELSE 'other' END AS file_id,
  CASE WHEN content_month_master.dl_rank_in_month <= 10 THEN content_master.min_dl_date ELSE NULL END AS min_dl_date,
  COUNT(DISTINCT(user_id)) AS cnt_user,
*/
  COUNT(DISTINCT(id)) AS cnt_dl,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl
JOIN
  content_master USING(file_id)
JOIN
  content_month_master USING(file_id)
WHERE
  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  1,2

""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_dl_rank = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_dl_rank.to_csv('df_dl_rank_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_dl_rank

<ipython-input-19-b29128674315>:67: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_dl_rank = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,dl_date,dl_rank_in_month,content_name,cnt_dl
0,2024-11-01,5,橘柊生バースデー動画(？)『DISH//'s BBQ』,75
1,2024-11-01,6,2024/10/27(日)北村匠海 監督映画「世界征服やめた」ワールドプレミア上映イベント,113
2,2024-11-01,7,2024.November,539
3,2024-11-01,8,2024.November,319
4,2024-11-01,9,2024/10/26(土)DJ To-i「SPOOKY PUMPKIN 2024 ～PURO...,58
...,...,...,...,...
258,2024-11-30,7,2024.November,16
259,2024-11-30,8,2024.November,6
260,2024-11-30,9,2024/10/26(土)DJ To-i「SPOOKY PUMPKIN 2024 ～PURO...,6
261,2024-11-30,10,【後編】泉大智バースデー動画『DISH//'sキッチン』,20


In [ ]:
##
query = """
WITH content_master AS (
SELECT
  file_id,
  content_type,
  content_name,
  MIN(DATE(downloaded_date)) OVER(PARTITION BY file_id) AS min_dl_date,
FROM
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}`
QUALIFY
  ROW_NUMBER() OVER(PARTITION BY file_id ORDER BY downloaded_date DESC, content_type, content_name) = 1
),

fc_mart AS (
SELECT
  system_id,
  sex,
  DATE(join_date) AS join_date,
  DATE_TRUNC(DATE(join_date), YEAR) AS join_year,
  birthday,
  CASE --年代別
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) < 0 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) > 100 THEN 'zz.Other'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 11 THEN 'a.~11'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 14 THEN 'b.12~14'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 17 THEN 'c.15~17'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 21 THEN 'd.18~21'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 29 THEN 'e.22~29'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 39 THEN 'f.30~39'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) <= 49 THEN 'g.40~49'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) >= 50 THEN 'h.50~'
    WHEN TRUNC(DATE_DIFF('{age_base_date}', birthday, MONTH) / 12) IS NULL THEN 'z.不明'
    ELSE 'zz.Other'
  END AS age_kubun,
FROM
  `gree-dionysus-stardust.production_sdr.process_fc_{group_name}_{current_month_start_date_nodash}`
WHERE
  DATE(join_date) <= '{previous_month_end_date}'
  AND DATE(membership_expiration_date) >= '{previous_month_start_date}'
)

SELECT
  file_id,
  content_master.content_type,
  content_master.content_name,
  content_master.min_dl_date,
  COUNT(DISTINCT(user_id)) AS cnt_user,
  COUNT(DISTINCT(id)) AS cnt_dl,
--性年代別
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'a.~11' THEN user_id ELSE NULL END)) AS uu_women_11,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'b.12~14' THEN user_id ELSE NULL END)) AS uu_women_12_14,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'c.15~17' THEN user_id ELSE NULL END)) AS uu_women_15_17,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'd.18~21' THEN user_id ELSE NULL END)) AS uu_women_18_21,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'e.22~29' THEN user_id ELSE NULL END)) AS uu_women_22_29,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'f.30~39' THEN user_id ELSE NULL END)) AS uu_women_30_39,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'g.40~49' THEN user_id ELSE NULL END)) AS uu_women_40_49,
  COUNT(DISTINCT(CASE WHEN sex = '1.女性' AND age_kubun = 'h.50~' THEN user_id ELSE NULL END)) AS uu_women_50_,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'a.~11' THEN user_id ELSE NULL END)) AS uu_women_11,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'b.12~14' THEN user_id ELSE NULL END)) AS uu_men_12_14,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'c.15~17' THEN user_id ELSE NULL END)) AS uu_men_15_17,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'd.18~21' THEN user_id ELSE NULL END)) AS uu_men_18_21,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'e.22~29' THEN user_id ELSE NULL END)) AS uu_men_22_29,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'f.30~39' THEN user_id ELSE NULL END)) AS uu_men_30_39,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'g.40~49' THEN user_id ELSE NULL END)) AS uu_men_40_49,
  COUNT(DISTINCT(CASE WHEN sex = '2.男性' AND age_kubun = 'h.50~' THEN user_id ELSE NULL END)) AS uu_men_50_,
  COUNT(DISTINCT(CASE WHEN sex NOT IN ('1.女性' ,'2.男性') OR age_kubun NOT LIKE '%.%' THEN user_id ELSE NULL END)) AS uu_otherage,
  COUNT(DISTINCT(CASE WHEN sex IS NULL OR age_kubun IS NULL THEN user_id ELSE NULL END)) AS uu_notFC,
  --FC入会年
  COUNT(DISTINCT(CASE WHEN join_year = '2014-01-01' THEN user_id ELSE NULL END)) AS uu_2014join,
  COUNT(DISTINCT(CASE WHEN join_year = '2015-01-01' THEN user_id ELSE NULL END)) AS uu_2015join,
  COUNT(DISTINCT(CASE WHEN join_year = '2016-01-01' THEN user_id ELSE NULL END)) AS uu_2016join,
  COUNT(DISTINCT(CASE WHEN join_year = '2017-01-01' THEN user_id ELSE NULL END)) AS uu_2017join,
  COUNT(DISTINCT(CASE WHEN join_year = '2018-01-01' THEN user_id ELSE NULL END)) AS uu_2018join,
  COUNT(DISTINCT(CASE WHEN join_year = '2019-01-01' THEN user_id ELSE NULL END)) AS uu_2019join,
  COUNT(DISTINCT(CASE WHEN join_year = '2020-01-01' THEN user_id ELSE NULL END)) AS uu_2020join,
  COUNT(DISTINCT(CASE WHEN join_year = '2021-01-01' THEN user_id ELSE NULL END)) AS uu_2021join,
  COUNT(DISTINCT(CASE WHEN join_year = '2022-01-01' THEN user_id ELSE NULL END)) AS uu_2022join,
  COUNT(DISTINCT(CASE WHEN join_year = '2023-01-01' THEN user_id ELSE NULL END)) AS uu_2023join,
  COUNT(DISTINCT(CASE WHEN join_year = '2024-01-01' THEN user_id ELSE NULL END)) AS uu_2024join,
  COUNT(DISTINCT(CASE WHEN join_year = '2025-01-01' THEN user_id ELSE NULL END)) AS uu_2025join,
  COUNT(DISTINCT(CASE WHEN NOT join_year BETWEEN '2014-01-01' AND '2025-01-01'THEN user_id ELSE NULL END)) AS uu_otherjoin,
  COUNT(DISTINCT(CASE WHEN join_year IS NULL THEN user_id ELSE NULL END)) AS uu_notFC,
FROM
  fc_mart
FULL OUTER JOIN
  `gree-dionysus-stardust.production_sdr.fc_viewing_data_{group_name}` content_dl ON fc_mart.system_id = content_dl.user_id
JOIN
  content_master USING(file_id)
WHERE
  DATE(downloaded_date) BETWEEN '{previous_month_start_date}' AND '{previous_month_end_date}'
GROUP BY
  ALL
ORDER BY
  cnt_dl DESC
LIMIT
  20


""".format(
    group_name=conf['group_name'],
    age_base_date=conf['age_base_date'],
    previous_month_start_date=conf['previous_month_start_date'],
    previous_month_end_date=conf['previous_month_end_date'],
    current_month_start_date=conf['current_month_start_date'],
    previous_month_start_date_nodash=conf['previous_month_start_date_nodash'],
    previous_month_end_date_nodash=conf['previous_month_end_date_nodash'],
    current_month_start_date_nodash=conf['current_month_start_date_nodash'],
)

df_detail = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')
df_detail.to_csv('df_detail_{group_name}.csv'.format(group_name=conf['group_name']), index=False)
df_detail

<ipython-input-20-02c317ea8809>:110: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(query, project_id='gree-dionysus-stardust', dialect='standard')


,file_id,content_type,content_name,min_dl_date,cnt_user,cnt_dl,uu_women_11,uu_women_12_14,uu_women_15_17,uu_women_18_21,...,uu_2017join,uu_2018join,uu_2019join,uu_2020join,uu_2021join,uu_2022join,uu_2023join,uu_2024join,uu_otherjoin,uu_notFC_1
0,c77b2732a7d3eec3d5e4b47b009335d40dc430e6e6bdf0...,新ムービー,2024/11/2(土)日本テレビ「with MUSIC」,2024-11-02,2663,4369,3,13,95,285,...,17,64,92,274,646,364,815,354,0,0
1,A1008533100000002642,フォト,DISH// ARENA LIVE 2023「HAPPY？」DAY2,2024-11-12,2145,3876,2,18,116,293,...,7,46,73,196,479,332,724,261,0,0
2,e8613e0c54c440578102994086b733bab01f90444e1a88...,新ムービー,2024/11/30(土)DISH// ARENA TOUR 2024-2025「群青飛行」...,2024-11-30,2228,2852,1,10,97,323,...,37,77,100,246,479,324,677,229,0,0
3,A1008533100000002643,フォト,DISH// ARENA LIVE 2023「HAPPY？」DAY1,2024-11-13,1335,2029,0,9,70,167,...,8,28,41,117,265,201,471,193,0,0
4,16460a2a1802504811d3a3030fbf7995f2694ec0ad031f...,新ムービー,橘柊生バースデー動画(？)『DISH//'s BBQ』,2024-10-15,1094,1668,2,5,45,179,...,10,40,47,131,261,129,307,149,0,0
5,A1008533100000002636,フォト,2024/10/27(日)北村匠海 監督映画「世界征服やめた」ワールドプレミア上映イベント,2024-10-28,1032,1309,4,7,58,112,...,3,9,13,71,210,149,387,186,0,0
6,A1008530100000002638,PC壁紙,2024.November,2024-11-01,676,978,2,10,21,63,...,0,14,18,63,160,99,224,90,0,0
7,A1008530100000002639,PC壁紙,2024.November,2024-11-01,454,564,1,5,15,63,...,1,12,14,41,96,73,149,58,0,0
8,A1008533100000002637,フォト,2024/10/26(土)DJ To-i「SPOOKY PUMPKIN 2024 ～PURO...,2024-10-28,479,539,2,5,32,67,...,1,4,10,32,87,72,189,83,0,0
9,transfar3118,新ムービー,【後編】泉大智バースデー動画『DISH//'sキッチン』,2024-07-24,219,384,0,1,11,34,...,4,9,13,23,52,27,49,35,0,0


## ZIPファイルダウンロード

In [ ]:
import glob
import zipfile

targets = []

for f in sorted(glob.glob("./*.csv")):
  target=f
targets.append(target)

for f in glob.glob("./*.csv"):
  targets.append(f)

compFile01 = zipfile.ZipFile('{}.zip'.format(conf['file_name']), 'w')
for y in targets:
  compFile01.write(y)
compFile01.close()

/usr/lib/python3.10/zipfile.py:1528: UserWarning: Duplicate name: 'df_join_year_dish.csv'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


In [ ]:
# zipファイルダウンロード

files.download('{}.zip'.format(conf['file_name']))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>